In [ ]:
!odsc conda install -s fspyspark32_p38_cpu_v2

In [ ]:
# Upgrade Oracle ADS to pick up the latest preview version to maintain compatibility with Oracle Cloud Infrastructure.
!pip install --pre --no-deps oracle-ads==2.9.0rc0

Oracle Data Science service sample notebook.

Copyright (c) 2022, 2023 Oracle, Inc. All rights reserved. Licensed under the [Universal Permissive License v 1.0](https://oss.oracle.com/licenses/upl).

***

# <font color="red">Feature store handling querying operations</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color="teal">Oracle Cloud Infrastructure Data Science Service.</font></p>

---
# Overview:
---
Managing many datasets, data sources, and transformations for machine learning is complex and costly. Poorly cleaned data, data issues, bugs in transformations, data drift, and training serving skew all lead to increased model development time and poor model performance. Feature store can be used to solve many of the problems becuase it provides a centralised way to transform and access data for training and serving time. Feature store helps define a standardised pipeline for ingestion of data and querying of data.This notebook demonstrates how to use feature store using a notebook spark session.

Compatible conda pack: [PySpark 3.2 and Feature Store](https://docs.oracle.com/iaas/data-science/using/conda-pyspark-fam.htm) for CPU on Python 3.8 (version 1.0)

## Contents:

- <a href="#concepts">1. Introduction</a>
- <a href='#pre-requisites'>2. Pre-requisites to Running this Notebook</a>
    - <a href='#setup'>2.1. Setup</a>
    - <a href='#policies'>2.2 Policies</a>
    - <a href='#prerequisites_authentication'>2.3 Authentication</a>
    - <a href='#prerequisites_variables'>2.4 Variables</a>
- <a href='#featurestore_querying'>3. Feature store querying</a>
    - <a href='#data_exploration'>3.1. Exploration of data in feature store</a>
    - <a href='#load_featuregroup'>3.2. Create feature store logical entities</a>
    - <a href='#explore_featuregroup'>3.3. Explore feature groups</a>
    - <a href='#select_subset_featuregroup'>3.4. Select subset of features</a>
    - <a href='#filter_featuregroup'>3.5. Filter feature groups</a>
    - <a href='#join_featuregroup'>3.6. Apply joins on feature groups</a>
    - <a href='#create_dataset'>3.7. Create dataset from multiple or one feature group</a>
    - <a href='#sql_query'>3.8. Free form sql query</a>
    - <a href='#featurestore_yaml'>3.9. Feature store Entities using YAML</a>
- <a href='#ref'>4. References</a>

---

**Important:**

Placeholder text for required values are surrounded by angle brackets that must be removed when adding the indicated content. For example, when adding a database name to `database_name = "<database_name>"` would become `database_name = "production"`.

---

<a id="concepts"></a>
# 1. Introduction

OCI Data Science feature store is a stack-based API solution that's deployed using OCI Resource Manager in your tenancy.

Review the following key terms to understand the Data Science feature store:


* **Feature Vector**: Set of feature values for any one primary or identifier key. For example, all or a subset of features of customer id ‘2536’ can be called as one feature vector.

* **Feature**: A feature is an individual measurable property or characteristic of a phenomenon being observed.

* **Entity**: An entity is a group of semantically related features. The first step a consumer of features would typically do when accessing the feature store service is to list the entities and the entities associated features. Or an entity is an object or concept that is described by its features. Examples of entities are customer, product, transaction, review, image, document, and so on.

* **Feature Group**: A feature group in a feature store is a collection of related features that are often used together in machine learning (ML) models. It serves as an organizational unit within the feature store for you to manage, version and share features across different ML projects. By organizing features into groups, data scientists and ML engineers can efficiently discover, reuse and collaborate on features reducing the redundant work and ensuring consistency in feature engineering.

* **Feature Group Job**: A feature group job is the processing instance of a feature group. Each feature group job  includes validation results and statistics results.

* **Dataset**: A dataset is a collection of features that are used together to either train a model or perform model inference.

* **Dataset Job**: dataset job is the processing instance of a dataset. Each dataset job includes validation results and statistics results.

<a id='pre-requisites'></a>
# 2. Pre-requisites to Running this Notebook

Notebook Sessions are accessible through the following conda environment: 

* **PySpark 3.2 and Feature Store Python 3.8 (fspyspark32_p38_cpu_v2)**

You can customize `fspyspark32_p38_cpu_v2`, publish it, and use it as a runtime environment for a Notebook session.


<a id='setup'></a>
### 2.1. Setup

<a id='setup_spark-defaults'></a>
### `spark-defaults.conf`

The `spark-defaults.conf` file is used to define the properties that are used by Spark. A templated version is installed when you install a Data Science conda environment that supports PySpark. However, you must update the template so that the Data Catalog metastore can be accessed. You can do this manually. However, the `odsc data-catalog config` commandline tool is ideal for setting up the file because it gathers information about your environment, and uses that to build the file.

The `odsc data-catalog config` command line tool needs the `--metastore` option to define the Data Catalog metastore OCID. No other command line option is needed because settings have default values, or they take values from your notebook session environment. Following are common parameters that you may need to override.

The `--authentication` option sets the authentication mode. It supports resource principal and API keys. The preferred method for authentication is resource principal, which is sent with `--authentication resource_principal`. If you want to use API keys, then use the `--authentication api_key` option. If the `--authentication` isn't specified, API keys are used. When API keys are used, information from the OCI configuration file is used to create the `spark-defaults.conf` file.

Object Storage and Data Catalog are regional services. By default, the region is set to the region your notebook session is running in. This information is taken from the environment variable, `NB_REGION`. Use the `--region` option to override this behavior.

The default location of the `spark-defaults.conf` file is `/home/datascience/spark_conf_dir` as defined in the `SPARK_CONF_DIR` environment variable. Use the `--output` option to define the directory where to write the file.

You need to determine what settings are appropriate for your configuration. However, the following works for most configurations and is run in a terminal window.

```bash
odsc data-catalog config --authentication resource_principal --metastore <metastore_id>
```
For more assistance, use the following command in a terminal window:

```bash
odsc data-catalog config --help
```

<a id='policies'></a>
### 2.2. Policies
This section covers the creation of dynamic groups and policies needed to use the service.

* [Data Flow Policies](https://docs.oracle.com/iaas/data-flow/using/policies.htm)
* [Data Catalog Metastore Required Policies](https://docs.oracle.com/en-us/iaas/data-catalog/using/metastore.htm)
* [Getting Started with Data Flow](https://docs.oracle.com/iaas/data-flow/using/dfs_getting_started.htm)
* [About Data Science Policies](https://docs.oracle.com/iaas/data-science/using/policies.htm)

<a id="prerequisites_authentication"></a>
### 2.3. Authentication
The [Oracle Accelerated Data Science SDK (ADS)](https://docs.oracle.com/iaas/tools/ads-sdk/latest/index.html) controls the authentication mechanism with the notebook session.<br>
To setup authentication use the ```ads.set_auth("resource_principal")``` or ```ads.set_auth("api_key")```.

In [ ]:
import ads
ads.set_auth(auth="resource_principal", client_kwargs={"fs_service_endpoint": "https://{api_gateway}/20230101"})

<a id="prerequisites_variables"></a>
### 2.4. Variables
To run this notebook, you must provide some information about your tenancy configuration. To create and run a feature store, you must specify a `<compartment_id>` and `<metastore_id>` which is the OCID of the Data Catalog metastore.

In [ ]:
import os

compartment_id = os.environ.get("NB_SESSION_COMPARTMENT_OCID")
metastore_id = "<metastore_id>"

<a id="featurestore_querying"></a>
# 3. Feature store querying
By default the **PySpark 3.2 and Feature Store Python 3.8** conda environment includes pre-installed [great-expectations](https://legacy.docs.greatexpectations.io/en/latest/reference/core_concepts/validation.html) library. In an ADS feature store module, you can either use the Python programmatic or YAML interface to define feature store entities. The joining functionality is heavily inspired by the APIs used by Pandas to merge, join or filter DataFrames. The APIs allow you to specify which features to select from which feature group, how to join them and which features to use in join conditions.

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="iteritems is deprecated")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import pandas as pd
from ads.feature_store.feature_store import FeatureStore
from ads.feature_store.feature_group import FeatureGroup
from ads.feature_store.model_details import ModelDetails
from ads.feature_store.dataset import Dataset
from ads.feature_store.common.enums import DatasetIngestionMode

from ads.feature_store.feature_group_expectation import ExpectationType
from great_expectations.core import ExpectationSuite, ExpectationConfiguration
from ads.feature_store.feature_store_registrar import FeatureStoreRegistrar

<a id="data_exploration"></a>
### 3.1. Exploration of data in feature store

In [ ]:
flights_df = pd.read_csv("https://objectstorage.us-ashburn-1.oraclecloud.com/p/hh2NOgFJbVSg4amcLM3G3hkTuHyBD-8aE_iCsuZKEvIav1Wlld-3zfCawG4ycQGN/n/ociodscdev/b/oci-feature-store/o/beta/data/flights/flights.csv")[['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']]
flights_df = flights_df.head(100)
flights_df.head()

In [ ]:
airports_df = pd.read_csv("https://objectstorage.us-ashburn-1.oraclecloud.com/p/hh2NOgFJbVSg4amcLM3G3hkTuHyBD-8aE_iCsuZKEvIav1Wlld-3zfCawG4ycQGN/n/ociodscdev/b/oci-feature-store/o/beta/data/flights/airports.csv")
airports_df.head()

In [ ]:
airlines_df = pd.read_csv("https://objectstorage.us-ashburn-1.oraclecloud.com/p/hh2NOgFJbVSg4amcLM3G3hkTuHyBD-8aE_iCsuZKEvIav1Wlld-3zfCawG4ycQGN/n/ociodscdev/b/oci-feature-store/o/beta/data/flights/airlines.csv")
airlines_df.head()

<a id="load_featuregroup"></a>
### 3.2. Create feature store logical entities

<a id="create_feature_store"></a>
#### 3.2.1 Feature Store
Feature store is the top level entity for feature store service

Call the ```.create()``` method of the Feature store instance to create a feature store.

In [ ]:
feature_store_resource = (
    FeatureStore().
    with_description("Data consisting of flights").
    with_compartment_id(compartment_id).
    with_display_name("flights details").
    with_offline_config(metastore_id=metastore_id)
)

In [ ]:
feature_store = feature_store_resource.create()
feature_store

#### 3.2.2 Entity
An entity is a group of semantically related features.

In [ ]:
entity = feature_store.create_entity(
    display_name="Flight details2",
    description="description for flight details"
)
entity

#### 3.2.3 Feature group
A feature group is an object that represents a logical group of time-series feature data as it is found in a datasource.

<a id="create_feature_group_flights"></a>
##### Flights Feature Group

Create feature group for flights

<div>
    <img src="https://objectstorage.us-ashburn-1.oraclecloud.com/p/hh2NOgFJbVSg4amcLM3G3hkTuHyBD-8aE_iCsuZKEvIav1Wlld-3zfCawG4ycQGN/n/ociodscdev/b/oci-feature-store/o/beta/_images/feature_group_flights.gif" width="700" height="350" />
</div>

In [ ]:
feature_group_flights = (
    FeatureGroup()
    .with_feature_store_id(feature_store.id)
    .with_primary_keys(["FLIGHT_NUMBER"])
    .with_name("flights_feature_group")
    .with_entity_id(entity.id)
    .with_compartment_id(compartment_id)
    .with_schema_details_from_dataframe(flights_df)
)

In [ ]:
feature_group_flights.create()

In [ ]:
feature_group_flights.show()

In [ ]:
feature_group_flights.materialise(flights_df)

<a id="create_feature_group_airport"></a>
##### Airport Feature Group

Create feature group for airport

In [ ]:
expectation_suite_airports = ExpectationSuite(
    expectation_suite_name="test_airports_df"
)
expectation_suite_airports.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_null",
        kwargs={"column": "IATA_CODE"},
    )
)
expectation_suite_airports.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "LATITUDE", "min_value": -1.0, "max_value": 1.0},
    )
)

expectation_suite_airports.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "LONGITUDE", "min_value": -1.0, "max_value": 1.0},
    )
)

In [ ]:
feature_group_airports = (
    FeatureGroup()
    .with_feature_store_id(feature_store.id)
    .with_primary_keys(["IATA_CODE"])
    .with_name("airport_feature_group")
    .with_entity_id(entity.id)
    .with_compartment_id(compartment_id)
    .with_schema_details_from_dataframe(airports_df)
    .with_expectation_suite(
        expectation_suite=expectation_suite_airports,
        expectation_type=ExpectationType.LENIENT,
     )
)

In [ ]:
feature_group_airports.create()

In [ ]:
feature_group_airports.materialise(airports_df)

In [ ]:
feature_group_airports.show()

<a id="create_feature_group_airlines"></a>
##### Airlines Feature Group

Create feature group for airlines

<div>
    <img src="https://objectstorage.us-ashburn-1.oraclecloud.com/p/hh2NOgFJbVSg4amcLM3G3hkTuHyBD-8aE_iCsuZKEvIav1Wlld-3zfCawG4ycQGN/n/ociodscdev/b/oci-feature-store/o/beta/_images/feature_groups.gif" width="700" height="350" />
</div>

In [ ]:
expectation_suite_airlines = ExpectationSuite(
    expectation_suite_name="test_airlines_df"
)
expectation_suite_airlines.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_null",
        kwargs={"column": "IATA_CODE"},
    )
)

In [ ]:
feature_group_airlines = (
    FeatureGroup()
    .with_feature_store_id(feature_store.id)
    .with_primary_keys(["IATA_CODE"])
    .with_name("airlines_feature_group")
    .with_entity_id(entity.id)
    .with_compartment_id(compartment_id)
    .with_schema_details_from_dataframe(airlines_df)
    .with_expectation_suite(
        expectation_suite=expectation_suite_airlines,
        expectation_type=ExpectationType.STRICT,
     )
)

In [ ]:
feature_group_airlines.create()

In [ ]:
feature_group_airlines.materialise(airlines_df)

In [ ]:
feature_group_airlines.show()

<a id="explore_featuregroup"></a>
### 3.3. Explore feature groups

In [ ]:
feature_group_flights.get_features_df()

In [ ]:
feature_group_airports.get_features_df()

In [ ]:
feature_group_airlines.get_features_df()

You can retrieve feature data in a DataFrame, that can either be used to train models.

In [ ]:
feature_group_flights.select().show()

In [ ]:
feature_group_airports.select().show()

In [ ]:
feature_group_airlines.select().show()

You can call the `get_statistics()` method of the feature group to fetch statistics for a specific ingestion job.You can use `to_pandas()` or `to_json()` to view the statistics.
You can visualize feature statistics with `to_viz()`

In [ ]:
feature_group_airlines.get_statistics().to_pandas()

In [ ]:
feature_group_flights.get_statistics().to_pandas()

In [ ]:
feature_group_airlines.get_statistics().to_viz()

You can call the `get_validation_output()` method of the FeatureGroup instance to fetch validation results for a specific ingestion job.

In [ ]:
feature_group_airlines.get_validation_output().to_pandas()

In [ ]:
feature_group_airlines.get_validation_output().to_summary()

<a id="select_subset_featuregroup"></a>
### 3.4. Select subset of features

In [ ]:
feature_group_airlines.select(['IATA_CODE']).show()

<a id="filter_featuregroup"></a>
### 3.5. Filter feature groups

In [ ]:
feature_group_airlines.filter(feature_group_airlines.IATA_CODE == "EV").show()

<a id="join_featuregroup"></a>
### 3.6. Apply joins on feature groups
As in Pandas, if the feature has the same name on both feature groups, then you can use the `on=[]` paramter. If they have different names, then you can use the `left_on=[]` and `right_on=[]` paramters:

In [ ]:
from ads.feature_store.common.enums import JoinType

query = (
    feature_group_flights.select()
    .join(feature_group_airlines.select(), left_on=['ORIGIN_AIRPORT'], right_on=['IATA_CODE'], join_type=JoinType.LEFT)
    .join(feature_group_airports.select(), left_on=['AIRLINE'], right_on=['IATA_CODE'], join_type=JoinType.LEFT)
)
query.show(5)

In [ ]:
query.to_string()

<a id="create_dataset"></a>
### 3.7. Create dataset from multiple or one feature group
A dataset is a collection of feature snapshots that are joined together to either train a model or perform model inference.

<div>
    <img src="https://objectstorage.us-ashburn-1.oraclecloud.com/p/hh2NOgFJbVSg4amcLM3G3hkTuHyBD-8aE_iCsuZKEvIav1Wlld-3zfCawG4ycQGN/n/ociodscdev/b/oci-feature-store/o/beta/_images/dataset.gif" width="700" height="350" />
</div>

In [ ]:
dataset = (
    Dataset()
    .with_description("Combined dataset for flights")
    .with_compartment_id(compartment_id)
    .with_name("flights_dataset")
    .with_entity_id(entity.id)
    .with_feature_store_id(feature_store.id)
    .with_query(query.to_string())
)

<a id="create_dataset"></a>
##### Create Dataset

Call the ```.create()``` method of the Dataset instance to create a dataset.

In [ ]:
dataset.create()

In [ ]:
dataset.materialise()

#### Interoperability with model

In [ ]:
model_details = ModelDetails().with_items(["ocid1.modelcatalog.oc1.unique_ocid"])
dataset.add_models(model_details)

<a id="visualise_lineage"></a>
#### Visualise lineage

Use the ```.show()``` method on the Dataset instance to visualize the lineage of the dataset.

In [ ]:
dataset.show()

In [ ]:
dataset.profile().show()

In [ ]:
dataset.as_of(version_number=0).show()

<a id="sql_query"></a>
### 3.8.  Freeform SQL query
Feature store provides a way to query feature store using free flow query. User need to mention `entity id` as the database name and `feature group name` as the table name to query feature store. This functionality can be useful if you need to express more complex queries for your use case

In [ ]:
entity_id = entity.id

sql = (f"SELECT flights_feature_group.*, airport_feature_group.IATA_CODE "
       f"FROM `{entity_id}`.flights_feature_group flights_feature_group "
       f"LEFT JOIN `{entity_id}`.airport_feature_group airport_feature_group "
       f"ON flights_feature_group.ORIGIN_AIRPORT=airport_feature_group.IATA_CODE")

In [ ]:
feature_store.sql(sql).show()

<a id="featurestore_yaml"></a>
### 3.9. Feature store Entities using YAML
In an ADS feature store module, you can either use the Python programmatic interface or YAML to define feature store entities. Below section describes how to create feature store entities using YAML as an interface.

In [ ]:
feature_store_yaml = """
apiVersion: v1
kind: featureStore
spec:
  displayName: Flights feature store
  compartmentId: "ocid1.tenancy.oc1..aaaaaaaa462hfhplpx652b32ix62xrdijppq2c7okwcqjlgrbknhgtj2kofa"
  offlineConfig:
    metastoreId: "ocid1.datacatalogmetastore.oc1.iad.amaaaaaabiudgxyap7tizm4gscwz7amu7dixz7ml3mtesqzzwwg3urvvdgua"

  entity: &flights_entity
    - kind: entity
      spec:
        name: Flights

  featureGroup:
    - kind: featureGroup
      spec:
        entity: *flights_entity
        name: flights_feature_group
        primaryKeys:
          - IATA_CODE
        inputFeatureDetails:
          - featureType: STRING
            name: IATA_CODE
            orderNumber: 1
          - featureType: STRING
            name: AIRPORT
            orderNumber: 2
          - featureType: STRING
            name: CITY
            orderNumber: 3
          - featureType: STRING
            name: STATE
            orderNumber: 4
          - featureType: STRING
            name: COUNTRY
            orderNumber: 5
          - featureType: FLOAT
            name: LATITUDE
            orderNumber: 6
          - featureType: FLOAT
            name: LONGITUDE
            orderNumber: 7
    - kind: featureGroup
      spec:
        entity: *flights_entity
        name: airlines_feature_group
        primaryKeys:
          - IATA_CODE
        inputFeatureDetails:
          - featureType: STRING
            name: IATA_CODE
            orderNumber: 1
          - featureType: STRING
            name: AIRPORT
            orderNumber: 2
          - featureType: STRING
            name: CITY
            orderNumber: 3
          - featureType: STRING
            name: STATE
            orderNumber: 4
          - featureType: STRING
            name: COUNTRY
            orderNumber: 5
          - featureType: FLOAT
            name: LATITUDE
            orderNumber: 6
          - featureType: FLOAT
            name: LONGITUDE
            orderNumber: 7

    - kind: featureGroup
      spec:
        entity: *flights_entity
        name: airport_feature_group
        primaryKeys:
          - IATA_CODE
        inputFeatureDetails:
          - featureType: STRING
            name: IATA_CODE
            orderNumber: 1
          - featureType: STRING
            name: AIRLINE
            orderNumber: 2
  dataset:
    - kind: dataset
      spec:
        name: flights_dataset
        entity: *flights_entity
        description: "Dataset for flights"
        query: 'SELECT flight.IATA_CODE, flight.AIRPORT FROM flights_feature_group flight'
"""

In [ ]:
registrar = FeatureStoreRegistrar.from_yaml(yaml_string=feature_store_yaml)
registrar.create()

<a id='ref'></a>
# 4. References

- [Feature Store Documentation](https://feature-store-accelerated-data-science.readthedocs.io/en/latest/overview.html)
- [ADS Library Documentation](https://accelerated-data-science.readthedocs.io/en/latest/index.html)
- [Data Science YouTube Videos](https://www.youtube.com/playlist?list=PLKCk3OyNwIzv6CWMhvqSB_8MLJIZdO80L)
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)